In [27]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Combine and Tidy Data Sources

In [28]:
djia = pd.read_csv("data/^DJI.csv")
confirmed = pd.read_csv("data/time_series_19-covid-Confirmed.csv")
deaths = pd.read_csv("data/time_series_19-covid-Deaths.csv")
recovered = pd.read_csv("data/time_series_19-covid-Recovered.csv")

In [29]:
djia = djia[djia['Date'] >= '2020-01-01']
djia['Date']=djia['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [30]:
# tidy 3 dfs
melt_ids = ['Province/State', 'Country/Region', 'Lat', 'Long']

tidy_confirmed = confirmed.melt(id_vars=melt_ids, 
                               value_vars=[c for c in confirmed.columns \
                                           if c not in ['Province/State', 'Country/Region', 'Lat', 'Long']], 
                               var_name='Date', value_name='Confirmed')

tidy_deaths = deaths.melt(id_vars=melt_ids, 
                               value_vars=[c for c in confirmed.columns \
                                           if c not in ['Province/State', 'Country/Region', 'Lat', 'Long']], 
                               var_name='Date', value_name='Deaths')

tidy_recovered = recovered.melt(id_vars=melt_ids, 
                                value_vars=[c for c in confirmed.columns \
                                           if c not in ['Province/State', 'Country/Region', 'Lat', 'Long']], 
                                var_name='Date', value_name='Recovered')


# merge 3 dfs
melt_ids.append('Date')
df = pd.merge(pd.merge(tidy_confirmed, tidy_deaths, on=melt_ids), 
                         tidy_recovered, on=melt_ids)

# tidy merged df
tidy_df = df.melt(id_vars=melt_ids, 
                  value_vars=['Confirmed', 'Deaths', 'Recovered'],
                  var_name='Type', 
                  value_name='Count')

# format date col
tidy_df['Date']=tidy_df['Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%y'))

## Plot

In [31]:
# counts by country/region
tidy_df = tidy_df.drop(['Lat', 'Long'],axis=1).groupby(['Country/Region', 'Type', 'Date']).sum().reset_index()

# counts by US/Non-US
tidy_df['US'] = tidy_df['Country/Region'] == 'US'
tidy_df['US'].replace(True, 'US', inplace=True)
tidy_df['US'].replace(False, 'Non-US', inplace=True)
tidy_df = tidy_df.groupby(['US', 'Date', 'Type']).sum().reset_index()

In [32]:
# merge djia 
tidy_df = tidy_df.merge(djia, how='left', on='Date')

In [33]:
tidy_df.sample(5)

,US,Date,Type,Count,Open,High,Low,Close,Adj Close,Volume
62,Non-US,2020-02-11,Recovered,4680,29390.710938,29415.390625,29210.470703,29276.339844,29276.339844,279540000.0
215,US,2020-02-16,Recovered,3,NaN,NaN,NaN,NaN,NaN,NaN
55,Non-US,2020-02-09,Deaths,906,NaN,NaN,NaN,NaN,NaN,NaN
194,US,2020-02-09,Recovered,3,NaN,NaN,NaN,NaN,NaN,NaN
10,Non-US,2020-01-25,Deaths,42,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
fig = make_subplots(rows=3, cols=2)
for i, case_type in enumerate(['Confirmed', 'Deaths', 'Recovered']):
    for j, loc in enumerate(['US','Non-US']):
        plot_df = tidy_df[(tidy_df['Type']==case_type)&(tidy_df['US']==loc)]
        fig.add_trace(
            go.Scatter(x=plot_df['Count'], y=plot_df['Adj Close'], 
                       mode='markers', 
                       name="{}-{}".format(loc,case_type)
            ),
            row=i+1, col=j+1
        )
        
fig.show()

In [35]:
plot_df = tidy_df[(tidy_df['Type']=='Confirmed')&(tidy_df['US']=='Non-US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Confirmed Non-US Cases')

In [36]:
plot_df = tidy_df[(tidy_df['Type']=='Recovered')&(tidy_df['US']=='Non-US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Recovered Non-US Cases')

In [37]:
plot_df = tidy_df[(tidy_df['Type']=='Deaths')&(tidy_df['US']=='Non-US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Deaths Non-US Cases')

In [38]:
plot_df = tidy_df[(tidy_df['Type']=='Confirmed')&(tidy_df['US']=='US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Confirmed US Cases')

In [39]:
plot_df = tidy_df[(tidy_df['Type']=='Recovered')&(tidy_df['US']=='US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Recovered US Cases')

In [40]:
plot_df = tidy_df[(tidy_df['Type']=='Deaths')&(tidy_df['US']=='US')]

px.scatter(plot_df, x='Count', y='Adj Close', title='DJIA vs Deaths US Cases')

In [60]:
loc = 'US'
case_type = 'Confirmed'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

In [61]:
loc = 'US'
case_type = 'Deaths'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

In [65]:
loc = 'US'
case_type = 'Recovered'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

In [62]:
loc = 'Non-US'
case_type = 'Confirmed'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

In [63]:
loc = 'Non-US'
case_type = 'Deaths'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

In [64]:
loc = 'Non-US'
case_type = 'Recovered'

plot_df = tidy_df[(tidy_df['US']==loc)&(tidy_df['Type']==case_type)]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Count'], mode='markers', name='# Cases'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='markers', name='DJIA Close'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=plot_df['Date'], y=plot_df['Open'], mode='markers', name='DJIA Open'),
    secondary_y=True,
)

fig.update_layout(
    title="{}, {}".format(case_type, loc)
)

## Sentiment Analysis

In [46]:
assert False

AssertionError: 

In [ ]:
from textblob import TextBlob
import sqlite3
import scipy.stats as stats

In [47]:
with sqlite3.connect('data/newspaper.db') as conn:
    articles = pd.read_sql("""
                            SELECT * FROM articles
                            WHERE date NOT NULL
                            AND text NOT NULL
                            """, conn)

In [23]:
articles['date'] = articles['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [24]:
articles = articles[~articles['text'].isna()]

In [26]:
px.scatter(articles, x='date', y='site')

In [ ]:
def sentiment_analysis(row):
    text = TextBlob(row['text'])
    
    row['polarity'] = text.sentiment.polarity
    row['subjectivity'] = text.sentiment.subjectivity
    
    return row

In [ ]:
articles = articles.apply(sentiment_analysis, axis=1)

In [ ]:
articles.sample(5)

In [ ]:
articles[articles['polarity']==articles['polarity'].min()]['text']

In [ ]:
sentiment = articles.groupby(['site', 'date'])['polarity'].describe().reset_index()

In [ ]:
alpha = 0.05

sentiment['sentiment_upper_bound'] = sentiment['mean'] + stats.t.ppf(1-(alpha/2), df=sentiment['count']-1)*sentiment['std']/np.sqrt(sentiment['count'])
sentiment['sentiment_lower_bound'] = sentiment['mean'] - stats.t.ppf(1-(alpha/2), df=sentiment['count']-1)*sentiment['std']/np.sqrt(sentiment['count'])

In [ ]:
sentiment.drop(['min', '25%', '50%', '75%', 'max', 'count', 'std'], axis=1, inplace=True)

In [ ]:
px.scatter(sentiment.melt(id_vars=['date'], value_vars=['sentiment_upper_bound', 'sentiment_lower_bound']),
           x='date',
           y='value',
           color='variable')

In [ ]:
sentiment.merge(tidy_df[tidy_df['Type']=='Confirmed'], left_on='date', right_on='Date')

In [ ]:
all_data = tidy_df[tidy_df['Type']=='Confirmed'].merge(sentiment, left_on='Date', right_on='date')

In [ ]:
all_data.drop(['Type', 'Open', 'High', 'Low', 'Close', 'Volume', 'date'], axis=1, inplace=True)

In [ ]:
all_data